In [ ]:
!pip install tensorflow_cpu

In [3]:
# supress the tensorflow warning messages
import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [4]:
import pandas as pd
import numpy as np
import keras
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [5]:
filepath = 'https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv'
concrete_data = pd.read_csv(filepath)
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [6]:
concrete_data.shape

(1030, 9)

In [7]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [8]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [10]:
concrete_date_columns = concrete_data.columns
concrete_date_columns

Index(['Cement', 'Blast Furnace Slag', 'Fly Ash', 'Water', 'Superplasticizer',
       'Coarse Aggregate', 'Fine Aggregate', 'Age', 'Strength'],
      dtype='object')

In [12]:
predictors = concrete_data[concrete_date_columns[concrete_date_columns != 'Strength']]
target = concrete_data['Strength']

In [15]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [16]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [17]:
predictor_norm = (predictors - predictors.mean()) / predictors.std()
predictor_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [18]:
predictor_norm.shape

(1030, 8)

In [19]:
n_cols = predictor_norm.shape[1]
n_cols

8

In [20]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [21]:
def regression_model():
    model = Sequential()
    model.add(Input(shape=(n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

In [22]:
model = regression_model()

In [23]:
model.fit(predictor_norm, target, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
23/23 - 1s - 44ms/step - loss: 1624.1595 - val_loss: 1110.4053
Epoch 2/100
23/23 - 0s - 4ms/step - loss: 1477.0725 - val_loss: 972.9045
Epoch 3/100
23/23 - 0s - 4ms/step - loss: 1253.5879 - val_loss: 776.0301
Epoch 4/100
23/23 - 0s - 4ms/step - loss: 933.4158 - val_loss: 528.5662
Epoch 5/100
23/23 - 0s - 4ms/step - loss: 578.6302 - val_loss: 310.6786
Epoch 6/100
23/23 - 0s - 4ms/step - loss: 341.1073 - val_loss: 201.0648
Epoch 7/100
23/23 - 0s - 4ms/step - loss: 257.6658 - val_loss: 172.9172
Epoch 8/100
23/23 - 0s - 4ms/step - loss: 236.5202 - val_loss: 167.8383
Epoch 9/100
23/23 - 0s - 4ms/step - loss: 219.9227 - val_loss: 164.5420
Epoch 10/100
23/23 - 0s - 4ms/step - loss: 208.0910 - val_loss: 162.6789
Epoch 11/100
23/23 - 0s - 4ms/step - loss: 199.5271 - val_loss: 160.5599
Epoch 12/100
23/23 - 0s - 4ms/step - loss: 191.7027 - val_loss: 159.6841
Epoch 13/100
23/23 - 0s - 4ms/step - loss: 184.6749 - val_loss: 158.5312
Epoch 14/100
23/23 - 0s - 4ms/step - loss: 179.4651 - v

In [25]:
def regression_model():
    model = Sequential()
    model.add(Input(shape=(n_cols, )))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

model = regression_model()
model.fit(predictor_norm, target, validation_split=0.1, epochs=100, verbose=2)

Epoch 1/100
29/29 - 1s - 43ms/step - loss: 1434.1406 - val_loss: 694.4140
Epoch 2/100
29/29 - 0s - 3ms/step - loss: 458.5903 - val_loss: 191.2146
Epoch 3/100
29/29 - 0s - 3ms/step - loss: 239.8417 - val_loss: 192.5884
Epoch 4/100
29/29 - 0s - 3ms/step - loss: 208.5367 - val_loss: 164.9790
Epoch 5/100
29/29 - 0s - 3ms/step - loss: 183.0183 - val_loss: 161.1631
Epoch 6/100
29/29 - 0s - 3ms/step - loss: 169.8352 - val_loss: 141.6115
Epoch 7/100
29/29 - 0s - 3ms/step - loss: 152.5392 - val_loss: 130.8346
Epoch 8/100
29/29 - 0s - 3ms/step - loss: 135.0187 - val_loss: 118.0668
Epoch 9/100
29/29 - 0s - 3ms/step - loss: 123.0330 - val_loss: 106.9865
Epoch 10/100
29/29 - 0s - 3ms/step - loss: 109.9979 - val_loss: 90.3121
Epoch 11/100
29/29 - 0s - 3ms/step - loss: 96.0247 - val_loss: 85.6565
Epoch 12/100
29/29 - 0s - 3ms/step - loss: 85.7843 - val_loss: 81.7024
Epoch 13/100
29/29 - 0s - 3ms/step - loss: 81.1903 - val_loss: 71.0395
Epoch 14/100
29/29 - 0s - 3ms/step - loss: 71.3859 - val_loss: 70